# Team ALOHA Code
0. 라이브러리 임포트
1. 데이터 불러오기
2. 필요 함수 정의
3. 데이터 전처리
4. 모델링
5. 참조
6. 강화학습

# 0. 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import FinanceDataReader as fdr
import pandas_datareader as pdr
import datetime
from dateutil.relativedelta import relativedelta
import plotly.express as px
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pytimekr import pytimekr
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import BDay

import warnings                 # warning 무시
warnings.filterwarnings('ignore')

import ssl    # Mac에서 FinanceDataReader가 안나오는 오류 해결
ssl._create_default_https_context = ssl._create_unverified_context

from matplotlib import rc       # 한글 깨짐 방지
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 1. 데이터 불러오기

In [ ]:
Jan=pd.read_csv("NASDAQ_RSS_IFO_202301.csv", encoding='CP949')
Feb=pd.read_csv("NASDAQ_RSS_IFO_202302.csv", encoding='CP949')
Mar=pd.read_csv("NASDAQ_RSS_IFO_202303.csv", encoding='CP949')
Apr=pd.read_csv("NASDAQ_RSS_IFO_202304.csv", encoding='CP949')
May=pd.read_csv("NASDAQ_RSS_IFO_202305.csv", encoding='CP949')
Jun=pd.read_csv("NASDAQ_RSS_IFO_202306.csv", encoding='CP949')
Jul=pd.read_csv("NASDAQ_RSS_IFO_202307.csv", encoding='CP949')
Aug=pd.read_csv("NASDAQ_RSS_IFO_202308.csv", encoding='CP949')

In [ ]:
df=pd.concat([Jan, Feb, Mar, Apr, May, Jun, Jul, Aug], axis=0).reset_index(drop=True)
df=df.drop_duplicates(subset=['rgs_dt', 'tck_iem_cd'], keep='first').reset_index(drop=True).dropna()
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y%m%d')

# 2. 함수

In [ ]:
# 시간 변환 함수 (days)
# Input // date: 기준 날짜(YYYY-MM-DD, dtype:int), days: 변경되는 days(dtype:int)
# Output // YYYY-MM-DD (dtype:str)

def time_delta_days(date, days):                                           # 시간 변환 함수
    if len(str(date))==8:
        dtend=datetime.datetime.strptime(str(date), '%Y%m%d')              # string -> datetime
    else:
        dtend=date
    result=dtend+datetime.timedelta(days=days)    
    return result                                                          # datetime

In [ ]:
# Input // df: dataframe iloc: df의 iloc(n, dtype:int), days1: 원하는 범위1(n, dtype:int), days1: 원하는 범위2(n, dtype:int)
# Output // 사건 발생 시점의 -days1 ~ +days2까지의 주식 데이터의 dataframe
def ReadingData(df, iloc, days1, days2):
    start=time_delta_days(df.iloc[iloc]['rgs_dt'],days1)
    end=time_delta_days(df.iloc[iloc]['rgs_dt'],days2)
    # print(df.iloc[iloc]['tck_iem_cd'], start+' ~ '+end)
    return pd.DataFrame(fdr.DataReader(df.iloc[iloc]['tck_iem_cd'], start=start, end=end).Close)

In [ ]:
# Input // df: f주식 데이터의 dataframe(dtype: DataFrame), date: 원하는 MovingAverage(n,dtype:int)
# Output // [가격, ~일 이동평균선]의 약 1년 DataFrame

def MA(df, date):
    rolling_mean=df['Close'].rolling(window = date).mean()
    rolling_mean.name = f'{date} MA'
    curve=pd.concat([df, rolling_mean], axis=1)
    return curve[-1:-250:-1].sort_index()

In [ ]:
# Input // 

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df['rgs_dt'].min(), end=df['rgs_dt'].max())
weekdays = pd.date_range(start=df['rgs_dt'].min(), end=df['rgs_dt'].max(), freq='B')

def replace_with_nearest_future_weekday(date):
    if date in holidays or date not in weekdays:
        next_weekday = date + pd.DateOffset(days=1)
        while next_weekday not in weekdays:
            next_weekday += pd.DateOffset(days=1)
        return next_weekday.date()
    else:
        return date

# 3. 데이터 전처리

## 3-1) 주말 및 공휴일 날짜를 평일 날짜로 변경

In [ ]:
# 'rgs_dt' 열의 날짜를 대체
df['rgs_dt'] = df['rgs_dt'].apply(replace_with_nearest_future_weekday)

## 3-1) Finance Data를 불러오지 못하는 기업 제거

In [ ]:
stop = len(df)
count=0
counter=0
Dlist=[]
Error_tick=[]
for i in range (0, stop):
    try:
        if len(ReadingData(df, i, -365, 0))>=10:
            pass
        else :
            condition=df["tck_iem_cd"]==df["tck_iem_cd"][i]
            df=df[~condition]
            df=df.reset_index().drop(columns='index')
            Dlist.append(i)
            counter+=1
    except Exception as e:
        condition=df["tck_iem_cd"]==df["tck_iem_cd"][i]
        df=df[~condition]
        df=df.reset_index().drop(columns='index')
        Dlist.append(i)
        Error_tick.append(df["tck_iem_cd"][i])
        counter+=1
    count+=1
    
    if count % 1000 == 0:
        print(count)
    
df=df[:stop]

In [ ]:
df.to_csv("train_1.csv", index=False)
df=pd.read_csv("train_1.csv").reset_index(drop=True)
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')

## 3-2) summary가 되어있지 않은 데이터 제거 및 감성분석

In [ ]:
drop_list=[]

# summary의 글자 수가 1000개 이상인 뉴스 제거
for i in range (0, len(df["news_smy_ifo"])):
    if len(df["news_smy_ifo"][i]) >= 1000:
        drop_list.append(i)
           
df=df.drop(drop_list).reset_index().drop(columns=["index"])

sentence_list=df["news_smy_ifo"].tolist()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

sentences = sentence_list
results = nlp(sentences)
sent=pd.concat([df,pd.DataFrame(results)], axis=1)
df=sent.copy()

## 3-3) summary의 불용어 제거 이후 clustering

In [ ]:
# 불용어 목록 다운로드

stop_words = set(stopwords.words('english'))  # 영어 불용어 목록을 사용하려면 'english'를 사용합니다.

corpus = df["news_smy_ifo"].tolist()
corpus_without_stopwords = []
for sentence in corpus:
    # 문장을 소문자로 변환
    sentence = sentence.lower()
    
    # 문장들의 문장부호 제거
    sentence = re.sub(r'[()\[\]{}!,@^_&#|]', '', sentence)
    
    # 문장을 단어로 토큰화
    words = word_tokenize(sentence)
    
    # 불용어를 제거하고 새로운 문장을 만듭니다.
    filtered_sentence = [word for word in words if word not in stop_words]
    
    # 다시 문장으로 변환
    filtered_sentence = ' '.join(filtered_sentence)
    
    corpus_without_stopwords.append(filtered_sentence)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

Key=[]

# 데이터 프레임에서 세부사업명 열을 추출하여 리스트로 변환
corpus2=corpus_without_stopwords.copy()

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus2)

# K-means 클러스터링
k = 29  # 클러스터 개수
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

# 클러스터링 결과를 데이터 프레임에 추가
df['cluster'] = kmeans.labels_

# 각 클러스터의 키워드 추출
top_keywords = []
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(k):
    cluster_keywords = [terms[ind] for ind in order_centroids[i, :3]]  # 상위 5개 키워드 추출
    top_keywords.append(cluster_keywords)

# 클러스터링 결과와 키워드 출력
for cluster_id in range(k):
    cluster_samples = df[df['cluster'] == cluster_id]
    keywords = ", ".join(top_keywords[cluster_id])
    Key.append(keywords)
    # print(f"Cluster {cluster_id}:")
    # print("Keywords:", keywords)
    # print("Samples:")
    # print(cluster_samples['cluster'])
    # print('————————————')
    
keydf=pd.DataFrame(np.array(Key), columns=["keywords"])
keydf.index.name='cluster'
keydf

## 3-4) alpha와 beta 계산

In [ ]:
# df=pd.read_csv("15146.csv")
# df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')
df=df[df['tck_iem_cd']!='Corporate']
df=df[df['tck_iem_cd']!='Fintech']
df=df[df['tck_iem_cd']!='INAQ']
df=df[df['tck_iem_cd']!='ATEN']
df=df[df['tck_iem_cd']!='GHRS']
df=df.reset_index(drop=True)
df

In [ ]:
# Importing libraries and packages
import statsmodels.api as sm
from statsmodels import regression

list_alpha=[]
list_beta=[]
error=[]
error_tick=[]
count=0

def linreg(x,y):
    x = sm.add_constant(x)
    model = regression.linear_model.OLS(y,x).fit()
    x = x[:, 1]
    return model.params[0], model.params[1]

asset_data1=ReadingData(df, 0, -365, 0).rename(columns={"Close":df["tck_iem_cd"][0]})
asset_data2=ReadingData(df, len(df)-1, -365, 0).rename(columns={"Close":df["tck_iem_cd"][len(df)-1]})
nasdaq=pd.DataFrame(fdr.DataReader('^IXIC', start=asset_data1.index[0], end=asset_data2.index[-1]).Close).rename(columns={'Close':'NASDAQ'})

for k in range(0, len(df)):
    try:
        asset_data = ReadingData(df, k, -365, 0).rename(columns={"Close": df["tck_iem_cd"][k]})
        merge = pd.concat([asset_data, nasdaq], axis=1).dropna().pct_change().dropna()

        # Regression model
        X = pd.DataFrame(merge["NASDAQ"]).values
        Y = merge.drop(columns=["NASDAQ"]).values

        alpha, beta = linreg(X, Y)

        list_alpha.append(alpha)
        list_beta.append(beta)
    except Exception as e:
        error.append(k)
        error_tick.append(df["tck_iem_cd"][k])
        print(f"Error for K={k}: {str(e)}")
        continue  # 오류가 발생해도 계속 진행
    count+=1
    
    if count % 1000 == 0:
        print(count)

In [ ]:
alphabeta=pd.DataFrame([list_alpha, list_beta]).T.rename(columns={0:'alpha', 1:'beta'})
df=pd.concat([df.drop(error).reset_index(drop=True), alphabeta], axis=1).dropna()
df

In [ ]:
# df.to_csv("28515_ab.csv", index=False)
df.to_csv("train_2.csv", index=False)

## 3-5) Industry 분류

In [ ]:
# df=pd.read_csv("28515_ab.csv").reset_index(drop=True)
df=pd.read_csv("train_2.csv").reset_index(drop=True)
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')

In [ ]:
industry = pd.read_csv("tck_industry.csv", index_col=0)
ind=industry.copy()
ind["industry"]=ind["IndustryCode"].apply(lambda x:x//1000000)
ind=ind.drop_duplicates(subset='Symbol', keep='first')
ind=ind[['Symbol','industry']].rename(columns={"Symbol":"tck_iem_cd"})

In [ ]:
df=pd.merge(df, ind, on='tck_iem_cd', how='left').dropna()

In [ ]:
# df.to_csv("TrainingData.csv", index=False)
df.to_csv("train_3.csv", index=False)

## 3-6) Target 추가

In [ ]:
# df=pd.read_csv("TrainingData.csv").reset_index(drop=True)
df=pd.read_csv("train_3.csv").reset_index(drop=True)
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')

In [ ]:
start=datetime.datetime.now()

MA_list=[]
counter=0
R=len(df)
# R=100
for n in range (0, R):
    data=ReadingData(df,n,-15,15)
    Data=MA(MA(MA(data,2),3),4).loc[:time_delta_days(df['rgs_dt'][n],4)]
    value=Data.iloc[-5:len(Data):4].pct_change().dropna().reset_index(drop=True)
    MA_list.append(value)
    counter+=1
    
    if counter%1000==0:
        end=datetime.datetime.now()
        print(counter, end-start)

In [ ]:
# 제거
df=df[df['tck_iem_cd']!='CETUU'].reset_index(drop=True)

In [ ]:
Concat=pd.concat(MA_list, axis=0).reset_index(drop=True)
Concat.index=np.arange(0, len(df))
Final=pd.concat([df,Concat], axis=1).dropna()

## 3-7) 4월 7일에 대한 데이터 수정

**4월 7일은 Good Friday로 미국 증권시장이 닫힌 날**

In [ ]:
df=Final
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')

In [ ]:
df.loc[df['rgs_dt'] == '2023-04-07', 'rgs_dt'] = '2023-04-10'

In [ ]:
start=datetime.datetime.now()

MA_list=[]
counter=0
R=len(df)

for n in range (7341, 7396):
    data=ReadingData(df,n,-15,15)
    Data=MA(MA(MA(data,2),3),4).loc[:time_delta_days(df['rgs_dt'][n],4)]
    value=Data.iloc[-5:len(Data):4].pct_change().dropna().reset_index(drop=True)
    MA_list.append(value)
    counter+=1
    
    if counter%1000==0:
        end=datetime.datetime.now()
        print(counter, end-start)

In [ ]:
Concat47=pd.concat(MA_list, axis=0).reset_index(drop=True)
Concat47.index=range (7341, 7396)
df47=df.iloc[7341:7396].drop(columns=['Close', '2 MA', '3 MA', '4 MA'])
drop47=df.drop(df.index[7341:7396])
Final47=pd.concat([df47,Concat47], axis=1)

In [ ]:
df=pd.concat([drop47, Final47], axis=0).sort_index()

## 3-8) 금리 추가

In [ ]:
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')

In [ ]:
start_date='2023-01-01'
end_date='2023-12-01'
rf=pdr.get_data_fred('FEDFUNDS', start_date, end_date)
stock_df=fdr.DataReader("AAPL", start=start_date, end=end_date)[['Close']]
fed_funds_df=rf.copy()
fed_funds_df=fed_funds_df.reset_index()
stock_df=stock_df.reset_index()

In [ ]:
# 날짜 열을 Datetime 형식으로 변환
fed_funds_df['DATE'] = pd.to_datetime(fed_funds_df['DATE'])
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# 월(Month) 열을 추출
fed_funds_df['Month'] = fed_funds_df['DATE'].dt.to_period('M')
stock_df['Month'] = stock_df['Date'].dt.to_period('M')

# 월별로 그룹화하여 월별 금리 데이터를 추가
merged_df = stock_df.copy()  # 결과를 저장할 DataFrame 복제

for month, group in stock_df.groupby('Month'):
    month_fed_funds = fed_funds_df[fed_funds_df['Month'] == month]
    if not month_fed_funds.empty:
        merged_df.loc[group.index, 'FEDFUNDS'] = month_fed_funds['FEDFUNDS'].values[0]

# 결과 출력
merged_df

In [ ]:
merged_df=merged_df.rename(columns={"Date":"rgs_dt", "FEDFUNDS":"R_f"})
merged_df=merged_df[['rgs_dt','R_f']]

In [ ]:
data8=pd.merge(df, merged_df, how='inner', on='rgs_dt')

## 3-9) 환율 추가

In [ ]:
df=data8
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')

In [ ]:
KRW=fdr.DataReader('KRW/USD',start=start_date)[['Close']].rename(columns={"Close":"KRW/USD"})
CNY=fdr.DataReader('CNY/USD',start=start_date)[['Close']].rename(columns={"Close":"CNY/USD"})
JPY=fdr.DataReader('JPY/USD',start=start_date)[['Close']].rename(columns={"Close":"JPY/USD"})
EUR=fdr.DataReader('EUR/USD',start=start_date)[['Close']].rename(columns={"Close":"EUR/USD"})

In [ ]:
ex_rate=pd.concat([KRW,CNY,JPY,EUR],axis=1)
ex_rate=ex_rate.reset_index().rename(columns={"Date":"rgs_dt"})

In [ ]:
data9=pd.merge(df, ex_rate, how='inner', on='rgs_dt')

## 3-9. MA 수정

In [ ]:
# label : 긍/부정+중립
df=data9
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')
df=df.drop(['Close', '2 MA', '3 MA', '4 MA'], axis=1)

In [ ]:
start=datetime.datetime.now()

ma_df_list=[]
counter=0

error=[]
error_tick=[]

for n in range (0, len(df)):
    try:
        data=ReadingData(df,n,-50,60)
        MovAvg=MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(MA(data,2),3),4),5),6),7),8),9),10),11),12),13),14),15),16),17),18),19),20),21),22),23),24),25),26),27),28),29),30).dropna()
        # MovAvg.loc[:time_delta_days(df['rgs_dt'][n],0)]
        # MovAvg.loc[time_delta_days(df['rgs_dt'][n],0):time_delta_days(df['rgs_dt'][n],30)]
        MAvg=MovAvg.loc[time_delta_days(df['rgs_dt'][n],0):]
        befMAvg=MovAvg.loc[:time_delta_days(df['rgs_dt'][n],0)].iloc[-2:-1:][0:1]
        befMAvg

        MA1_after=MAvg.loc[time_delta_days(df['rgs_dt'][n],1-1):]['Close'].reset_index(drop=True)
        for m in range (2, 8):
            exec(f"MA{m}_after = MAvg.loc[time_delta_days(df['rgs_dt'][n],{m}-1):]['{m} MA'].reset_index(drop=True)")

        MovingA=pd.concat([MA1_after, MA2_after, MA3_after, MA4_after, MA5_after, MA6_after ,MA7_after], axis=1).dropna()
        MovingA=MovingA[:30]

        MA_lists = [[] for _ in range(7)]
        MA_lists[0]=((MovingA['Close']-befMAvg['Close'][0])/befMAvg['Close'][0])
        for i in range (1, 7):
            MA_lists[i]=((MovingA[f'{i+1} MA']-befMAvg[f'{i+1} MA'][0])/befMAvg[f'{i+1} MA'][0])
        MA_df=pd.DataFrame(MA_lists).T.iloc[:30].rename(columns={'Close':'1 MA'})

        LIST=[]
        for M in range (1, 8):
            for iloc in range(0, len(MA_df)):
                LIST.append(MA_df.iloc[iloc][f'{M} MA'])
        LIST2=[]
        ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
        for M in range (1, 8):
            for iloc in range(0, len(MA_df)):
                LIST2.append(f'{M} MA '+ordinal(iloc+1))
        ma_df=pd.DataFrame(LIST).T
        ma_df.columns=LIST2
        
        ma_df_list.append(ma_df)
    except Exception as e:
        error.append(n)
        error_tick.append(df["tck_iem_cd"][n])
        print(f"Error for K={n}: {str(e)}")
        continue  # 오류가 발생해도 계속 진행
    counter+=1
    
    if counter%1000==0:
        end=datetime.datetime.now()
        print(counter, end-start)

In [ ]:
#최종 Train Data
df=df.drop(error).reset_index(drop=True)
MA_DF=pd.concat(ma_df_list).reset_index(drop=True)
final_data=pd.concat([df,MA_DF], axis=1)
final_data.to_csv("Final_Train_Data.csv", index=False)

# 4. 모델링

## 4-1. 전처리

In [ ]:
data=pd.read_csv('Final_Train_Data.csv')
df=data.copy()

#불필요 열 삭제
df=df.drop(['rgs_dt', 'tck_iem_cd', 'til_ifo', 'ctgy_cfc_ifo', 'mdi_ifo', 'news_smy_ifo', 'rld_ose_iem_tck_cd', 'url_ifo', 'KRW/USD', 'JPY/USD'], axis=1)

#컬럼명 변경
df=df.rename(columns={'label': 'sent', 
                   'score': 'conf', 
                   'cluster': 'event',
                    'R_f': 'ir'})

event_column = df.pop('event')
df['event'] = event_column

#컬럼 순서 변경
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]

#컬럼 분리
cn = df.columns.tolist()
feature, MA1, MA2, MA3, MA4, MA5, MA6, MA7=cn[0:9], cn[9:39], cn[39:69], cn[69:99], cn[99:129], cn[129:159], cn[159:189], cn[189:219]

#결측치 삭제
df=df.drop(2172)

#데이터타입 변경
df.industry=df.industry.astype('int')

## 4-2. 그리드서치를 통한 하이퍼파라미터 최적화 및 성능테스트

In [ ]:
#feature, target 지정
features=df[feature]
features = features.iloc[:, [0, 1, 5, 2, 3, 4, 6, 7, 8]]
target = df[MA1[0]]

#인코딩
categorical = ['industry','event','sent']
for category in categorical:
    features[category] = features[category].astype('category')
    features = pd.get_dummies(features, columns=[category], prefix=category, dtype='int')

#train/test 데이터 분리 8:2로
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=50)

In [ ]:
#그리드서치

# 각 파라미터 값 넣기
xgboost_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

xgboost = xgb.XGBRegressor(n_jobs=-1)

grid_cv = GridSearchCV(estimator=xgboost, param_grid=xgboost_params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)

print(f'최적 하이퍼 파라미터: {grid_cv.best_params_}')

In [ ]:
# 최적 파라미터 설정
best_xgboost_params = grid_cv.best_params_

xgboost = xgb.XGBRegressor(
    n_estimators=best_xgboost_params['n_estimators'], 
    max_depth=best_xgboost_params['max_depth'], 
    learning_rate=best_xgboost_params['learning_rate'],
    n_jobs=-1
)

#학습
xgboost.fit(X_train, y_train)

xgboost_pred = xgboost.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#성능 테스트 함수
def evaluate_xgboost_regressor(model, X, y):
    # 모델로 예측
    y_pred = model.predict(X)
    
    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    
    # MAE 계산
    mae = mean_absolute_error(y, y_pred)
    
    # R-squared 계산
    r_squared = r2_score(y, y_pred)
    
    # 예측값과 실제값의 오차값 리스트 생성
    error_list = y - y_pred
    
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R-squared:", r_squared)
    
# 함수 사용 예시
evaluate_xgboost_regressor(xgboost, X_test, y_test)

## 4-3. 모델링 및 결과 저장

In [ ]:
MAs=[MA1, MA2, MA3, MA4, MA5, MA6, MA7]

In [ ]:
X_train=df[feature]
X_train = X_train.iloc[:, [0, 1, 5, 2, 3, 4, 6, 7, 8]]

events=pd.Series(X_train.iloc[:,0].unique()).sort_values().tolist()
sents=pd.Series(X_train.iloc[:,1].unique()).sort_values().tolist()
industries=pd.Series(X_train.iloc[:,2].unique()).sort_values().tolist()

events, sents, industries = [str(x) for x in events], [str(x) for x in sents], [str(x) for x in industries]

In [ ]:
pred_data = pd.DataFrame(columns=X_train.columns)

for event in events:
    for sent in sents:
        for industry in industries:
            new_row=[event, sent, industry, df.conf.mean(), df.alpha.mean(), df.beta.mean(), df.ir.mean(), df['CNY/USD'].mean(), df['EUR/USD'].mean()]
            pred_data = pred_data.append(pd.Series(new_row, index=X_train.columns), ignore_index=True)
            
result=pred_data.copy()

In [ ]:
count=0

for MA in MAs:
    for n in MA:
        
        if count==0:
            #X_train 생성 및 인코딩
            X_train=df[feature]
            X_train = X_train.iloc[:, [0, 1, 5, 2, 3, 4, 6, 7, 8]]

            categorical = ['industry','event','sent']
            for category in categorical:
                X_train[category] = X_train[category].astype('category')
                X_train = pd.get_dummies(X_train, columns=[category], prefix=category, dtype='int') 

            # 최적 파라미터 설정
            best_xgboost_params = grid_cv.best_params_

            model = xgb.XGBRegressor(
                n_estimators=best_xgboost_params['n_estimators'], 
                max_depth=best_xgboost_params['max_depth'], 
                learning_rate=best_xgboost_params['learning_rate'],
                n_jobs=-1
            )
        
        y_train = df[n]

        #학습
        model.fit(X_train, y_train)

        if count==0:
            #결과용 데이터 인코딩 및 컬럼 정리
            categorical = ['industry','event','sent']
            for category in categorical:
                pred_data[category] = pred_data[category].astype('category')
                pred_data = pd.get_dummies(pred_data, columns=[category], prefix=category, dtype='int')

            pred_data = pred_data[X_train.columns]
            
            count+=1
    
        model_pred = model.predict(pred_data)
        model_pred = list(model_pred)
        result[n] = model_pred

In [ ]:
#결과 저장
result.to_csv('final_result.csv', index=False)

## 4-4. 결과 해석

In [ ]:
interpretation=result.copy()

#industry re-labeling
interpretation['industry']=interpretation['industry'].astype(int)
interpretation['industry'].replace({0: 'Services', 
                       1: 'Others',
                       50: 'fossil fuel', 
                       51: 'chemical/mineral', 
                       52: 'heavy', 
                       53: 'accommodation/distribution', 
                       54: 'consumer/food',
                       55: 'finance', 
                       56: 'medical/pharmaceutical', 
                       57: 'semiconductor/IT', 
                       59: 'eco-friendly energy', 
                       60: 'real estate', 
                       62: 'legal',
                       63: 'edu'
                       }, inplace=True)

#event re-labeling
interpretation['event']=interpretation['event'].astype(int)
'''
interpretation['event'].replace({0: 'guru stock report', 
                       1: 'Fintel share report',
                       2: 'Fintel coverage recommendation', 
                       3: 'quarterly loss', 
                       4: 'recent fluctuations in shares',
                       5: 'nasdaq trading', 
                       6: 'performance and option', 
                       7: 'quarterly share and Zacks', 
                       8: 'share price fluctuations compared to moving average', 
                       9: 'stock fluctuations', 
                       10: 'quarterly revenue report', 
                       11: 'performance reccomandation', 
                       12: 'Tech stocks', 
                       13: 'Investors',  
                       14: 'dividend channel',  
                       15: 'declaring dividends by board',  
                       16: 'recommandation coverage report',  
                       17: 'investors and stock report',  
                       18: 'Chicago related news',  
                       19: 'stock recommendation',  
                       20: 'consumer and financial stocks',  
                       21: 'guru financial report',  
                       22: 'option trading',  
                       23: 'health care stock',  
                       24: 'NASDAQ100, RTT news info',  
                       25: 'Fintel reports',  
                       26: 'buy recommandation',  
                       27: 'year-to-date stock report',  
                       28: 'Others',  
                       }, inplace=True)
'''
interpretation['event'].replace({4: 'recent fluctuations in shares',  }, inplace=True)

In [ ]:
# Feature Importance 도출

ftr=interpretation.iloc[:, :9]
for category in categorical:
    ftr[category] = ftr[category].astype('category')
    ftr = pd.get_dummies(ftr, columns=[category], prefix=category, dtype='int') 

FI = pd.DataFrame({'Feature': ftr.columns})
FI['Importance'] = pd.Series(model.feature_importances_)
FI=FI.sort_values(by='Importance', ascending=False).reset_index(drop=True)
FI

In [ ]:
#예시로 MA=3으로 설정한 데이터 가져오기

df_MA3=interpretation.copy()
df_MA3=df_MA3[['event', 'sent', 'industry'] + MA3]

#열 이름 변경하기
import re
pattern = r'3 MA (\d+)'

# 열 이름 변경 함수
def rename_columns(col_name):
    match = re.match(pattern, col_name)
    if match:
        return match.group(1)  # 정규식 패턴에 맞는 숫자 부분을 반환
    else:
        return col_name

# 열 이름 변경 적용
df_MA3.columns = df_MA3.columns.map(rename_columns)

df_MA3

In [ ]:
import matplotlib.pyplot as plt

#MA 수익률 시각화 함수
#input: 산업, 이벤트
def MAER_Visualization(df, industry, event): 
    
    df=df[df['industry']==industry] #산업 선택
    df=df[df['event']==event]     #이벤트 선택

    data_to_plot_1 = df.iloc[0, 3:] #부정적 이벤트
    data_to_plot_2 = df.iloc[1, 3:] #중립적 이벤트
    data_to_plot_3 = df.iloc[2, 3:] #긍정적 이벤트

    plt.plot(data_to_plot_1, color='red', label='Negative')
    plt.plot(data_to_plot_2, color='black', label='Neutral')
    plt.plot(data_to_plot_3, color='blue', label='Positive')

    title='Changes in MAER when articles about '+event+' are published regarding the '+industry+' industry'
    industry+' 산업에 '+event+' 관련 기사가 났을 때의 이동평균 변화 추이'

    # 그래프 제목과 축 레이블 설정 (옵션)
    plt.title(title)
    plt.xlabel('time')
    plt.ylabel('MA Earning Rate')

    # 범례 추가
    plt.legend()

    # 그래프 표시
    plt.show()
    
    
#예시
MAER_Visualization(df_MA3, 'semiconductor/IT', 'recent fluctuations in shares')

# 5. 참조

## 5-1. Elbow Method (text clustering: k 결정하기)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import FinanceDataReader as fdr
import pandas_datareader as pdr
import datetime
from dateutil.relativedelta import relativedelta
import plotly.express as px
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pytimekr import pytimekr
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import BDay

import warnings                 # warning 무시
warnings.filterwarnings('ignore')

import ssl    # Mac에서 FinanceDataReader가 안나오는 오류 해결
ssl._create_default_https_context = ssl._create_unverified_context

from matplotlib import rc       # 한글 깨짐 방지
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

#데이터 불러오기
df=pd.read_csv("Final_TrainData.csv").reset_index(drop=True)
df['rgs_dt'] = pd.to_datetime(df['rgs_dt'], format='%Y-%m-%d')
df=df.drop(columns=['cluster'])

#함수 정의
from sklearn.cluster import KMeans
def visualize_elbowmethod(data, N, param_init='random', param_n_init=10, param_max_iter=300):
    distortions = []
    for i in range(1, N):
        km = KMeans(n_clusters=i, init=param_init, n_init=param_n_init, max_iter=param_max_iter, random_state=0)
        km.fit(data)
        distortions.append(km.inertia_)

    plt.plot(range(1, N), distortions, marker='o')
    plt.xlabel('Number of Cluster')
    plt.ylabel('Distortion')
    plt.show()
    
from sklearn.metrics import silhouette_samples, silhouette_score
def visualize_silhouette_layer(data, N, param_init='random', param_n_init=10, param_max_iter=300):
    clusters_range = range(2,N+1)
    results = []

    for i in clusters_range:
        clusterer = KMeans(n_clusters=i, init=param_init, n_init=param_n_init, max_iter=param_max_iter, random_state=0)
        cluster_labels = clusterer.fit_predict(data)
        silhouette_avg = silhouette_score(data, cluster_labels)
        results.append([i, silhouette_avg])

    result = pd.DataFrame(results, columns=["n_clusters", "silhouette_score"])
    pivot_km = pd.pivot_table(result, index="n_clusters", values="silhouette_score")

    plt.figure()
    sns.heatmap(pivot_km, annot=True, linewidths=.5, fmt='.3f', cmap=sns.cm._rocket_lut)
    plt.tight_layout()
    plt.show()
    
# 불용어 목록 다운로드
stop_words = set(stopwords.words('english'))  # 영어 불용어 목록을 사용하려면 'english'를 사용합니다.

corpus = df["news_smy_ifo"].tolist()
corpus_without_stopwords = []
for sentence in corpus:
    # 문장을 소문자로 변환
    sentence = sentence.lower()
    
    # 문장들의 문장부호 제거
    sentence = re.sub(r'[()\[\]{}!,@^_&#|]', '', sentence)
    
    # 문장을 단어로 토큰화
    words = word_tokenize(sentence)
    
    # 불용어를 제거하고 새로운 문장을 만듭니다.
    filtered_sentence = [word for word in words if word not in stop_words]
    
    # 다시 문장으로 변환
    filtered_sentence = ' '.join(filtered_sentence)
    
    corpus_without_stopwords.append(filtered_sentence)

#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist


Key=[]

# 데이터 프레임에서 세부사업명 열을 추출하여 리스트로 변환
corpus2=corpus_without_stopwords.copy()

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus2)

#시각화
visualize_elbowmethod(X, 40)

# 5-2. Industry 산업 분류

In [ ]:
ind=pd.read_csv("tck_industry.csv")

indCode=df['industry'].unique().tolist()

for code in indCode:
    A=ind0[ind0['IndustryCode']//1000000==code]
    print(code,":")
    print(A['Industry'].unique())

# 6. 강화학습

In [ ]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def load_data(industry):
    episode_list = []
    episode_name = []
    path = os.getcwd()
    tck_path = '{}/Data/{}'.format(path,industry)
    
    tck_list = os.listdir(tck_path)
    for tck in tck_list:
        
        event_path = f'{path}/Data/{industry}/{tck}'
        event_list = os.listdir(event_path)
        
        for event in event_list:
            os.chdir(event_path)
            df = pd.read_csv(event, dtype={'code': str}, parse_dates=['Date'])
            df['action'] = 0
            df = df.drop(columns=['Date'])

            episode_list.append(df)
            
    return episode_list

# 각 이벤트를 리스트에 저장

In [ ]:
def plot_price_chart(data, buy_signals, sell_signals):
    plt.figure(figsize=(10, 6))
    plt.plot(data['Adj Close'], label='Price', alpha=0.7)

    # Correcting the scatter function
    if buy_signals:
        buy_x, buy_y = zip(*buy_signals)
        plt.scatter(buy_x, buy_y, color='g', marker='^', label='Buy Signal')

    if sell_signals:
        sell_x, sell_y = zip(*sell_signals)
        plt.scatter(sell_x, sell_y, color='r', marker='v', label='Sell Signal')

    plt.title('Price Chart with Buy/Sell Signals')
    plt.xlabel('Time Step')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

def plot_portfolio_values(portfolio_values):
    plt.plot(portfolio_values)
    plt.title('Portfolio Value Over Time')
    plt.xlabel('Time Step')
    plt.ylabel('Portfolio Value')
    plt.show()

In [ ]:
def I_MR(ma_ratio):
    if ma_ratio > 1:
        return -1
    elif ma_ratio <= 1:
        return 1



In [ ]:
class TradingEnvironment:
    def __init__(self, episode_data):
        self.episode_data = episode_data
        self.current_step = 0
        self.max_steps = len(episode_data)
        self.transaction_cost = 0.01  # Adjust this based on your scenario
        self.initial_balance = 1000000  # Adjust this based on your scenario
        self.balance = self.initial_balance
        self.num_stocks = 0
        self.portfolio_value = self.balance

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.num_stocks = 0
        self.portfolio_value = self.balance


    def get_state(self):
        state = self.episode_data.iloc[self.current_step].to_dict()

        # Convert non-numerical values to numbers if possible
        for key, value in state.items():
            if key != 'Date':
                try:
                    state[key] = float(value)
                except (ValueError, TypeError):
                    # Use a default value or skip the key if conversion is not possible
                    state[key] = 0.0  # You can change this default value as needed

        return state

    def decide_buying_unit(self,ma_ratio, max_buyable_stocks):
        if ma_ratio <=0.6:
            return max(1, int(max_buyable_stocks))
        elif ma_ratio <=0.8:
            return max(1, int(0.75*int(max_buyable_stocks)))
        else:
            return max(1, int(0.2*int(max_buyable_stocks)))
        
    def decide_selling_unit(self,ma_ratio, max_sellable_stocks):
        if ma_ratio >=1.4:
            return max(1, int(max_sellable_stocks))
        elif ma_ratio <=1.2:
            return max(1, int(0.75*int(max_sellable_stocks)))
        else:
            return max(1, int(0.2*int(max_sellable_stocks)))
    
    
    def take_action(self, action):
        current_price = self.episode_data.iloc[self.current_step]['Adj Close']

        # Previous action
        prev_action = 0 if self.current_step == 0 else self.episode_data.iloc[self.current_step - 1]['action']

        # Transaction cost
        transaction_cost = int(prev_action != action) * self.transaction_cost 

        # Moving Average Ratio
        ma_ratio = int(self.episode_data.iloc[self.current_step]['MA_ratio'])
        # Reward calculation
        
        
        if action == 0:  # Sell
            if self.num_stocks > 0:  # Ensure there are stocks to sell
                max_sellable_stocks = self.num_stocks
                trading_unit = min(max_sellable_stocks, self.decide_selling_unit(ma_ratio,max_sellable_stocks))
                sell_amount = current_price * (1 - self.transaction_cost) * trading_unit
                self.balance += sell_amount
                self.num_stocks -= trading_unit
            reward = (action-1) * I_MR(ma_ratio) - transaction_cost
            
        elif action ==1 : # Hold
            if ma_ratio > 1.2:
                reward = 0.8
            elif ma_ratio >= 0.8:
                reward = 1.2
            elif ma_ratio < 0.8:
                reward = 0.8
        elif action == 2:  # Buy
            max_buyable_stocks = int(self.balance / (current_price * (1 + self.transaction_cost)))
            trading_unit = min(max_buyable_stocks, self.decide_buying_unit(ma_ratio,max_buyable_stocks))
            buy_amount = current_price * (1 + self.transaction_cost) * trading_unit

            if self.balance >= buy_amount:  # Ensure there is enough balance to buy
                self.balance -= buy_amount
                self.num_stocks += trading_unit
            reward = (action-1) * I_MR(ma_ratio) - transaction_cost

        self.portfolio_value = self.balance + current_price * self.num_stocks
        self.current_step += 1
        done = self.current_step == self.max_steps - 1
        if self.current_step == len(self.episode_data)-1:
            done = True

        else:
            done = False
            
        return reward, done

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size, gamma=0.7, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.current_step = 0

        # Q-networks
        self.q_network = QNetwork(state_size, action_size)
        self.target_network = QNetwork(state_size, action_size)
        self.target_network.load_state_dict(self.q_network.state_dict())  # Copy initial weights

        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=0.001)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        state_values = [value for key, value in state.items()]
        state_tensor = torch.FloatTensor(state_values).unsqueeze(0)

        q_values = self.q_network(state_tensor)
        return torch.argmax(q_values).item()

    def train(self, state, action, reward, next_state, done):
        state_values = [value for key, value in state.items()]
        state_tensor = torch.FloatTensor(state_values).unsqueeze(0)
        
        next_state_values = [value for key, value in next_state.items()]
        next_state_tensor = torch.FloatTensor(next_state_values).unsqueeze(0)
        
        action = torch.LongTensor([action])
        reward = torch.FloatTensor([reward])
        done = torch.FloatTensor([int(done)])

        # Q-value prediction for the current state
        q_values = self.q_network(state_tensor).gather(1, action.unsqueeze(1))

        # Q-value prediction for the next state
        next_q_values = self.target_network(next_state_tensor).detach().max(1)[0].unsqueeze(1)
        target = reward + (1 - done) * self.gamma * next_q_values

        # Compute the Huber loss
        loss = nn.functional.smooth_l1_loss(q_values, target)

        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

        # Update target network weights every few steps (e.g., every 10 episodes)
        if self.current_step % 10 == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())

        self.current_step += 1

In [ ]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)  
        self.fc4 = nn.Linear(hidden_size, action_size)   

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)  
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x


        
        
        


In [ ]:
episode_list = load_data('Industry_50')

# 데이터 분할
train_episodes, test_episodes = train_test_split(episode_list, test_size=0.05, random_state=42)

# Initialize the agent

state_size = 13
action_size = 3  
agent = DQNAgent(state_size=state_size, action_size=action_size)


train_num_episodes = len(train_episodes)
for train_episode_data in train_episodes:
    train_env = TradingEnvironment(train_episode_data)
    train_env.reset()
    total_reward = 0
    portfolio_values = []

    for t in range(29):
        train_state = train_env.get_state()
        train_action = agent.act(train_state)
        train_reward, train_done = train_env.take_action(train_action)
        train_next_state = train_env.get_state()
        agent.train(train_state, train_action, train_reward, train_next_state, train_done)

        total_reward += train_reward
        portfolio_values.append(train_env.portfolio_value)
#         print(f"Step {t}: Action - {train_action}, Reward - {train_reward}, Portfolio Value - {train_env.portfolio_value}")


#     print(f"Total Reward: {total_reward}")
#     print(f"Final Portfolio Value: {train_env.portfolio_value}")




# Testing loop
test_num_episodes = len(test_episodes)
for test_episode_data in test_episodes:
    test_env = TradingEnvironment(test_episode_data)
    test_env.reset()
    total_reward = 0
    portfolio_values = []

    # Lists to store buy and sell signals
    buy_signals = []
    sell_signals = []
    hold_postion = []

    for t in range(29):
        test_state = test_env.get_state()
        test_action = agent.act(test_state)
        test_reward, test_done = test_env.take_action(test_action)
        test_next_state = test_env.get_state()

        total_reward += test_reward

        portfolio_values.append(test_env.portfolio_value)
#         print(f"Step {t}: Action - {test_action}, Reward - {test_reward}, Portfolio Value - {test_env.portfolio_value}")

        # Record buy and sell signals
        if test_action == 2:  # Buy (long)
            buy_signals.append((t, test_episode_data.iloc[t]['Adj Close']))
        elif test_action == 0:  # Sell (short)
            sell_signals.append((t, test_episode_data.iloc[t]['Adj Close']))

    # Print  the performance metrics
    print(f"Total Reward: {total_reward}")
    print(f"Final Portfolio Value: {test_env.portfolio_value}")
    
#     plot_price_chart(train_episode_data, buy_signals, sell_signals)
    plot_portfolio_values(portfolio_values)
